# Deep Q-learning for Cart-Pole

This notebook uses OpenAI Gym and Deep Q-learning to creating a playing agent for Cart-Pole. 

### Import dependencies and create a Cart-Pole playing environment

In [1]:
import gym
import tensorflow as tf
import numpy as np

In [2]:
env = gym.make('CartPole-v0')

### Explore the OpenAI Gym environment

Get a list of the possible actions for this game

In [3]:
env.action_space

Discrete(2)

There are two possible actions, moving the cart left and right--coded as 0 or 1 in the environment

---

Let's run a random simulation to see how the game it played

In [4]:
env.reset()
rewards = []
for move in range(100):
    env.render()
    state, reward, done, info = env.step(env.action_space.sample())
    rewards.append(reward)
    if done:
        rewards = []
        env.reset()

In [5]:
env.close()

In [6]:
print(rewards)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


The object of the game is to move the cart left or right to keep the pole from falling. The longer the pole stays up, the more reward we receive. For this game, we get a reward of 1 for each step that the pole is still standing.

### Building the Q-Network

In reinforcement learning we usually keep a matrix of all state-action pairs and update the values to help the agent learn. For some games, such as cart-pole, the number of state-action paris is simply too large for this to be feasible. Even for a simple game like cart-pole, there are four real-valued numbers that make up each possible state--position and velocity of the cart, and position and velocity of the pole. This creates a nearly infinite number of states.

In deep Q-learning, we use a neural network to approximate the Q-table. Our A-network takes a state as input and outputs q-values for each possible action. 

Our targets for training are $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$, thus we want to minimize $(\hat{Q}(s,a) - Q(s,a))^2$. This can be thought of as a measurement of how much reward can be expected in the next time step if we take a given action.

In [ ]:
class QNetwork():
    def __init__(self, learning_rate=0.01, state_size=4, action_size=2, hidden_size=10, name='QNetwork'):
        with tf.variable_scope(name):
            self.inputs_ = tf.placeholder(tf.float32, [None, state_size], name='inputs')
            self.actions_ = tf.placeholder(tf.int32, [None], name='actions')
            one_hot_actions = tf.one_hot(self.actions_, action_size)

            # Target placeholder for training
            self.targetQs_ = tf.placeholder(tf.float32, [None], name='target')
            
            # Hidden layers
            self.fc1 = tf.contrib.layers.fully_connected(self.inputs_, hidden_size)
            self.fc2 = tf.contrib.layers.fully_connected(self.fc1, hidden_size)
            
            # Output layer
            self.output = tf.contrib.layers.fully_connected(self.fc2, action_size, activation_fn=None)
            
            # Trian on (targetQ - Q)^2
            self.Q = tf.reduce_sum(tf.multiply(self.output, one_hot_actions), axis=1)
            
            self.loss = tf.reduce_mean(tf.square(self.targetQs_ - self.Q))
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

### State Memory 
Reinforcement learning algorithms can have stability issues due to correlations between states. Thus, it's usually not a good idea to train in sequential states as the agent plays the game. Instead, we will let the agent play the game, store the experiences in memory, and then train the network on a random sample of past experiences.

In [14]:
from collections import deque

class Memory():
    def __init__(self, max_size=1000):
        self.buffer = deque(maxlen=max_size)
        
    def add(self, experience):
        self.buffer.append(experience)
        
    def sample(self, batch_size):
        index_list = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[index] for index in index_list]

### Exploration vs. Exploitation
In order for the agent to learn, it needs to explore its environmnet by taking random actions. As the agent learns, we want to take advantage of its exploration early on and choose what it thinks is the best action (exploit). At each step in the game we will decide whether the agent will explore or exploit. At the start of the game exploration will be more likely, but as the game progresses we will push the agent to exploit more.

### Training Algorithm
The network will be trained in *episodes*, which is the same as one simulation of the game. For Cart-Pole, the goal of an episode is to keep the pole upright for 195 frames. We start a new episode when meeting that goal or if the game ends because the pole tilts too far or the cart tries to move off the screen. This is how we'll train the agent.

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

### Hyperparameters

In [15]:
train_episodes = 1000          # max number of episodes to learn from
max_steps = 200                # max steps in an episode
gamma = 0.99                   # future reward discount

# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
hidden_size = 64               # number of units in each Q-network hidden layer
learning_rate = 0.0001         # Q-network learning rate

# Memory parameters
memory_size = 10000            # memory capacity
batch_size = 20                # experience mini-batch size
pretrain_length = batch_size   # number experiences to pretrain the memory

In [16]:
tf.reset_default_graph()
mainQN = QNetwork(name='main', hidden_size=hidden_size, learning_rate=learning_rate)

### Initialize Memory
Reset the simulation and populate the memory with a set of transitions to train on. 

In [20]:
# Initialize the simulation
env.reset()
# Take one random step to get the pole and cart moving
state, reward, done, _ = env.step(env.action_space.sample())

memory = Memory(max_size=memory_size)

for index in range(pretrain_length):
    # Uncomment to watch simulation
    # env.render()
    
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    
    if done:
        next_state = np.zeros(state.shape)
        memory.add((state, action, reward, next_state))
        env.reset()
        state, reward, done, _ = env.step(env.action_space.sample())
    else:
        memory.add((state, action, reward, next_state))
        state = next_state

### Training

In [ ]:
saver = tf.train.Saver()
rewards_list = []
loss = 0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    total_steps = 0
    for episode in range(1, train_episodes):
        total_reward = 0
        current_step = 0
        while current_step < max_steps:
            total_steps += 1
            
            # Determine whether to explore or exploit
            explore_p = explore_stop + (explore_start - explore_stop)*np.exp(-decay_rate*total_steps)
            if explore_p > np.random.rand():
                # Explore a random action
                action = env.action_space.sample()
            else:
                # Exploit best action
                feed = {mainQN.inputs_: state.reshape((1, *state.shape))}
                Qs = sess.run(mainQN.output, feed_dict=feed)
                action = np.argmax(Qs)
                
            # Execute action to get new state and reward
            next_state, reward, done, _ = env.step(action)
            
            total_reward += reward
            
            if done:
                # the episode ends so no next state
                next_state = np.zeros(state.shape)
                current_step = max_steps
                
                print('Episode: {}'.format(episode),
                      'Total reward: {}'.format(total_reward),
                      'Training loss: {:.4f}'.format(loss),
                      'Explore P: {:.4f}'.format(explore_p))
                rewards_list.append((episode, total_reward))
                
                # Add experience to memory
                memory.add((state, action, reward, next_state))
                
                # Start new episode
                env.reset()
                # Take a random step to get the cart moving
                state, reward, done, _ = env.step(env.action_space.sample())
                
            else:
                # Add experience to memory
                memory.add((state, action, reward, next_state))
                state - next_state
                current_step += 1
                
            # Everything above is just to fill out the memory and let the agent play the game, this is where we actually train
            batch = memory.sample(batch_size)
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            rewards = np.array([each[2] for each in batch])
            next_states = np.array([each[3] for each in batch])
            
            target_Qs = sess.run(mainQN.output, feed_dict={mainQN.inputs_: next_states})
            # Set target_Qs to 0 for states where episode ends
            episode_ends = (next_states == np.zeros(states[0].shape)).all(axis=1)
            target_Qs[episode_ends] = (0, 0)
            
            targets = rewards + gamma * np.max(target_Qs, axis=1)
            
            loss, _ = sess.run([mainQN.loss, mainQN.optimizer],
                              feed_dict={mainQN.inputs_: states,
                                        mainQN.targetQs_: targets,
                                        mainQN.actions_: actions})
            
    saver.save(sess, "cartpole.ckpt")
    

Episode: 1 Total reward: 13.0 Training loss: 0.9675 Explore P: 0.9987
Episode: 2 Total reward: 17.0 Training loss: 0.9660 Explore P: 0.9970
Episode: 3 Total reward: 20.0 Training loss: 0.9708 Explore P: 0.9951
Episode: 4 Total reward: 41.0 Training loss: 1.0424 Explore P: 0.9910
Episode: 5 Total reward: 21.0 Training loss: 1.0693 Explore P: 0.9890
Episode: 6 Total reward: 15.0 Training loss: 1.1583 Explore P: 0.9875
Episode: 7 Total reward: 14.0 Training loss: 1.0468 Explore P: 0.9861
Episode: 8 Total reward: 15.0 Training loss: 1.2851 Explore P: 0.9847
Episode: 9 Total reward: 20.0 Training loss: 1.2862 Explore P: 0.9827
Episode: 10 Total reward: 19.0 Training loss: 1.3787 Explore P: 0.9809
Episode: 11 Total reward: 48.0 Training loss: 1.3711 Explore P: 0.9762
Episode: 12 Total reward: 17.0 Training loss: 2.0318 Explore P: 0.9746
Episode: 13 Total reward: 53.0 Training loss: 3.0323 Explore P: 0.9695
Episode: 14 Total reward: 21.0 Training loss: 2.9219 Explore P: 0.9675
Episode: 15 Tot

Episode: 115 Total reward: 29.0 Training loss: 762.5773 Explore P: 0.7910
Episode: 116 Total reward: 20.0 Training loss: 113.4879 Explore P: 0.7895
Episode: 117 Total reward: 12.0 Training loss: 2938.4565 Explore P: 0.7885
Episode: 118 Total reward: 23.0 Training loss: 380.3663 Explore P: 0.7867
Episode: 119 Total reward: 42.0 Training loss: 1827.1836 Explore P: 0.7835
Episode: 120 Total reward: 19.0 Training loss: 738.6549 Explore P: 0.7820
Episode: 121 Total reward: 25.0 Training loss: 1556.6135 Explore P: 0.7801
Episode: 122 Total reward: 39.0 Training loss: 210.2809 Explore P: 0.7771
Episode: 123 Total reward: 32.0 Training loss: 2026.9443 Explore P: 0.7746
Episode: 124 Total reward: 9.0 Training loss: 854.8688 Explore P: 0.7740
Episode: 125 Total reward: 11.0 Training loss: 1472.1055 Explore P: 0.7731
Episode: 126 Total reward: 14.0 Training loss: 917.5771 Explore P: 0.7720
Episode: 127 Total reward: 10.0 Training loss: 1327.7900 Explore P: 0.7713
Episode: 128 Total reward: 24.0 T

Episode: 229 Total reward: 15.0 Training loss: 1052.1327 Explore P: 0.6427
Episode: 230 Total reward: 14.0 Training loss: 1217.5291 Explore P: 0.6418
Episode: 231 Total reward: 10.0 Training loss: 1698.3070 Explore P: 0.6412
Episode: 232 Total reward: 9.0 Training loss: 849.6009 Explore P: 0.6406
Episode: 233 Total reward: 24.0 Training loss: 1700.3365 Explore P: 0.6391
Episode: 234 Total reward: 21.0 Training loss: 1563.9998 Explore P: 0.6378
Episode: 235 Total reward: 24.0 Training loss: 184.1069 Explore P: 0.6363
Episode: 236 Total reward: 14.0 Training loss: 1826.1389 Explore P: 0.6354
Episode: 237 Total reward: 20.0 Training loss: 794.8887 Explore P: 0.6342
Episode: 238 Total reward: 9.0 Training loss: 558.7684 Explore P: 0.6336
Episode: 239 Total reward: 20.0 Training loss: 970.0158 Explore P: 0.6324
